In [1]:
import pandas as pd
import numpy as np
import os
import re
from copy import deepcopy 

from collections import Counter
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
base_dir = 'data/english_only/prompting_results_clean/with_metrics/' #experiment 1

In [3]:
from aggregate_metrics_helper import *

In [4]:
os.chdir('/shared/0/projects/research-jam-summer-2024/')
pd.options.mode.copy_on_write = True

In [137]:
base_dir_human = '/home/akananth/Misc/human-llm-similarity/src/annotation/annotation_output/'
f_human = 'annotated_instances_with_metrics_orig.jsonl'
df = pd.read_json(base_dir_human+f_human, orient='records', lines=True)
df.shape

(1272, 55)

In [139]:
end = df[(~df['HasResponse:::[1] Yes'].isna()) | (~df['HasResponse:::[2] No'].isna())]
end['annotated_end'] = 1 - ((~end['HasResponse:::[1] Yes'].isna()) & 
                            (~end.annotated_turn_3.isna())).astype(int)
end['human_end'] = 1 - ((end.human_turn_3!='[no response]') & (end.human_turn_3.apply(len)>0)).astype(int)
end.to_json(base_dir_human+re.sub('.jsonl','_end.jsonl',f_human), orient='records', lines=True)
end.shape

(1088, 57)

In [140]:
from sklearn.metrics import f1_score
f1_sample = pd.concat([
    end[end['human_end']==1],
    end[end['human_end']==0].sample(int(sum(end['human_end']==1)/1.6954177897574123)),
])
print(np.mean(f1_sample['human_end']))
print(f1_score(f1_sample['human_end'], f1_sample['annotated_end']))
f1_sample.shape

0.6292004634994206
0.5979591836734693


(863, 57)

In [154]:
pd.crosstab(f1_sample['human_end'], f1_sample['annotated_end'], normalize=0)

annotated_end,0,1
human_end,,
0,0.550000,0.450000
1,0.460405,0.539595


In [141]:
f1_sample[['user','instance_id','annotated_turn_3','human_turn_3','human_end','annotated_end']].\
to_csv('data/agg_metrics/human_baseline_end.csv', index=None)

In [143]:
base_dir_human = '/home/akananth/Misc/human-llm-similarity/src/annotation/annotation_output/'
f_human = 'annotated_instances_with_metrics.jsonl'
metrics = end[(end.human_end==0) & (end.annotated_end==0)]
metrics.to_json(base_dir_human+f_human, orient='records', lines=True)
metrics.shape

(293, 57)

In [144]:
metrics.to_csv('data/agg_metrics/human_baseline_metrics.csv', index=None)

In [115]:
base_dir_human = '/home/akananth/Misc/human-llm-similarity/src/annotation/annotation_output/'
f_human = 'annotated_instances_with_metrics.jsonl'
metrics = make_human_vs_llm_df(f_human, base_dir_human, read_dep = False, 
                               sim_prefix='annotated_', turn_3_prefix='')
metrics.columns = metrics.columns.str.replace('annotated_','llm_')
metrics = metrics[~metrics.llm_word_length.isna()]

annotated_instances_with_metrics.jsonl
read metrics
read embeddings


In [116]:
# log scale heavy-tailed count metrics
for k in ['word_count', 'word_length', 'perplexity', 'dep_dpth', 'dep_brth', 'dep_dep_dist']:
    metrics['human_'+k] = np.log(metrics['human_'+k]+1)
    metrics['llm_'+k] = np.log(metrics['llm_'+k]+1)


In [117]:
# check that columns are exactly the merge keys and human/llm metrics from all_metrics 
print(Counter(['human_'+k in metrics.columns for k in all_metrics]))
print(Counter(['llm_'+k in metrics.columns for k in all_metrics]))
print(Counter([k in metrics.columns for k in merge_keys]))
print(Counter([k in merge_keys or re.sub('human_|llm_','',k) in all_metrics 
               for k in metrics.columns]))


Counter({True: 20})
Counter({True: 20})
Counter({False: 3, True: 1})
Counter({True: 41})


In [120]:
# CREATE COLUMN AGGREGATES WITH CORRELATION
def corr_metric(corr_method, var = 'model', human_prefix='human_', sim_prefix='llm_'):
    vals = []
    metric = []
    cor = []
    for lvl in set(metrics[var]):
        print(lvl)
        sub = metrics[metrics[var] == lvl]
        for k in all_metrics:
            #print(k)
            vals.append(lvl)
            metric.append(k)
            cor.append(col_diff_correlate(sub[human_prefix+k], sub[sim_prefix+k], all_metrics[k], corr_method))
    col_corr = pd.DataFrame({var: vals, 'metric': metric, 'cor': cor, 'corr_method': corr_method})
    col_corr['category'] = col_corr['metric'].replace(metric_category)
    return col_corr

In [122]:
col_corr = {m: corr_metric(m, var='model') 
            for m in ['pearson','spearman','kendall']}


annotated_instances_with_metrics
annotated_instances_with_metrics
annotated_instances_with_metrics


In [124]:
col_corr['pearson'].to_csv('data/agg_metrics/cor_metrics_human_baseline.csv', index=None)

In [160]:
col_corr['pearson']

,model,metric,cor,corr_method,category
0,annotated_instances_with_metrics,word_count,0.168497,pearson,lexical
1,annotated_instances_with_metrics,word_length,0.149804,pearson,lexical
2,annotated_instances_with_metrics,perplexity,0.033057,pearson,lexical
3,annotated_instances_with_metrics,typo,0.247590,pearson,lexical
4,annotated_instances_with_metrics,pos,0.136363,pearson,syntactic
5,annotated_instances_with_metrics,dep_dpth,0.160833,pearson,syntactic
6,annotated_instances_with_metrics,dep_brth,0.213394,pearson,syntactic
7,annotated_instances_with_metrics,dep_dep_dist,0.237395,pearson,syntactic
8,annotated_instances_with_metrics,sbert,-0.006130,pearson,semantic
9,annotated_instances_with_metrics,liwc,0.082560,pearson,semantic
